In [1]:
import numpy as np
from gurobipy import Model, GRB
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.losses import MeanSquaredError

In [ ]:
def generar_instancia_aleatoria():

    c_x = np.random.randint(1, 10) 
    c_y = np.random.randint(1, 10)  
    c_t = np.random.uniform(1, 5)    

    a1 = np.random.randint(1, 4)
    b1 = np.random.randint(1, 4)
    c1 = np.random.randint(1, 10) 
    a2 = np.random.uniform(0.5, 3) 
    c2 = np.random.randint(0, 5)   

    return {
        "c_x": c_x,
        "c_y": c_y,
        "c_t": c_t,
        "a1": a1,
        "b1": b1,
        "c1": c1,
        "a2": a2,
        "c2": c2
    }

In [ ]:
def resolver_instancia(parametros):

    model = Model("Modelo MIP Aleatorio")

    x = model.addVar(vtype=GRB.BINARY, name="x")  
    y = model.addVar(vtype=GRB.BINARY, name="y") 
    t = model.addVar(vtype=GRB.CONTINUOUS, lb=0, name="t") 

    c_x = parametros["c_x"]
    c_y = parametros["c_y"]
    c_t = parametros["c_t"]
    a1 = parametros["a1"]
    b1 = parametros["b1"]
    c1 = parametros["c1"]
    a2 = parametros["a2"]
    c2 = parametros["c2"]

    model.setObjective(c_x * x + c_y * y + c_t * t, GRB.MAXIMIZE)

    model.addConstr(x + y <= 1, name="Restriccion_1")                # x + y <= 1
    model.addConstr(t - a1 * x - b1 * y <= c1, name="Restriccion_2") # t - a1*x - b1*y <= c1
    model.addConstr(t + a2 * x >= c2, name="Restriccion_3")          # t + a2*x >= c2

    model.setParam('OutputFlag', 0)

    model.optimize()

    if model.status == GRB.OPTIMAL:
        return {
            "valor_objetivo": model.objVal,
            "x": x.x,
            "y": y.x,
            "t": t.x,
            "parametros": parametros
        }
    else:
        return {
            "mensaje": "No se encontró una solución óptima.",
            "parametros": parametros
        }

In [ ]:
def generar_muestras_csv(n_muestras, filename):

    data = []
    for _ in range(n_muestras):
        parametros = generar_instancia_aleatoria()
        data.append(parametros)

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"{n_muestras} muestras generadas y guardadas en {filename}.")

#generar_muestras_csv(10000, "muestras.csv")


10000 muestras generadas y guardadas en muestras.csv.


In [ ]:
def resolver_y_guardar_con_filtro(filename_in, filename_out):

    df = pd.read_csv(filename_in)
    resultados = []

    for idx, fila in df.iterrows():
        parametros = fila.to_dict()
        resultado = resolver_instancia(parametros)
        
        if resultado.get("valor_objetivo") is not None: 
            resultados.append({
                "x": resultado["x"],
                "y": resultado["y"],
                "t": resultado["t"],
                "valor_objetivo": resultado["valor_objetivo"],
                **parametros
            })
        else: 
            resultados.append({
                "x": None,
                "y": None,
                "t": None,
                "valor_objetivo": None,
                **parametros
            })

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(filename_out, index=False)
    print(f"Resultados guardados en {filename_out}, incluyendo escenarios sin solución.")

#resolver_y_guardar_con_filtro("muestras.csv", "resultados_con_filtro.csv")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-17
Resultados guardados en resultados_con_filtro.csv, incluyendo escenarios sin solución.


In [ ]:

df = pd.read_csv("resultados_con_filtro.csv")

df_filtrado = df.dropna(subset=["valor_objetivo"])

X = df_filtrado.drop(columns=["x", "y", "t", "valor_objetivo"]) 
y = df_filtrado[["x", "y", "t", "valor_objetivo"]]           

scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

modelo = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(y_train.shape[1])
])

modelo.compile(optimizer="adam", loss="mse", metrics=["mae"])

historial = modelo.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

resultados = modelo.evaluate(X_test, y_test)
print(f"Pérdida: {resultados[0]}, MAE: {resultados[1]}")

#modelo.save("modelo_optimizacion.keras") # descomentar pero ya hay un modelo



Epoch 1/50


c:\Users\jjacq\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


249/249 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6253 - mae: 0.6462 - val_loss: 0.1651 - val_mae: 0.3000
Epoch 2/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1570 - mae: 0.2860 - val_loss: 0.1152 - val_mae: 0.2371
Epoch 3/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1107 - mae: 0.2258 - val_loss: 0.0810 - val_mae: 0.1833
Epoch 4/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0856 - mae: 0.1858 - val_loss: 0.0647 - val_mae: 0.1574
Epoch 5/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0701 - mae: 0.1598 - val_loss: 0.0563 - val_mae: 0.1385
Epoch 6/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0596 - mae: 0.1403 - val_loss: 0.0487 - val_mae: 0.1275
Epoch 7/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0502 - mae: 0.1266 - val_loss: 0.0413 - val_mae: 0.1140
Epoch 8/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0435 - mae: 0.1152 - val_loss: 0.0408 - val_mae: 0.1106
Epoch 9/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0355 

In [7]:
modelo = tf.keras.models.load_model("modelo_optimizacion.keras", custom_objects={"mse": MeanSquaredError()})

In [ ]:
num_nuevas_instancias = len(X_test)
num_aceptable = 0

for i in range(num_nuevas_instancias):
    nueva_muestra = X_test[i].reshape(1, -1)
    y_real = y_test[i].reshape(1, -1)
    y_real = scaler_y.inverse_transform(y_real)

    prediccion = modelo.predict(nueva_muestra, verbose=0)
    prediccion_desescalada = scaler_y.inverse_transform(prediccion)

    x_pred, y_pred, t_pred, valor_objetivo_pred = prediccion_desescalada[0]
    x_pred = 1 if x_pred > 0.5 else 0
    y_pred = 1 if y_pred > 0.5 else 0

    resultado_optimo = {
        'x': y_real[0][0],
        'y': y_real[0][1],
        't': y_real[0][2]
    }

    if (abs(x_pred - resultado_optimo['x']) <= 0.01 and
        abs(y_pred - resultado_optimo['y']) <= 0.01 and
        abs(t_pred - resultado_optimo['t'])/(resultado_optimo['t'] + 1e-8) <= 0.01):
        num_aceptable += 1



In [9]:
print(f"Soluciones aceptables: {num_aceptable} de {num_nuevas_instancias}, lo que representa un {num_aceptable/num_nuevas_instancias:.2%} de precisión.")

Soluciones aceptables: 1188 de 1990, lo que representa un 59.70% de precisión.
